# Deliverable 3. Create a Travel Itinerary Map

----

1. Create a folder called `Vacation_Itinerary` to store all the files for this deliverable.

2. Download the `Vacation_Itinerary_starter_code.ipynb` file into your `Vacation_Itinerary` folder and rename it `Vacation_Itinerary.ipynb`.

3. Make sure the initial dependencies and the Geoapify API key are imported.

4. From your `Vacation_Search` folder from Deliverable 2, import the `WeatherPy_vacation.csv` file as a DataFrame named `vacation_df`.

5. Use GeoViews to create a map that shows all the cities in the `vacation_df` DataFrame. Configure the map as follows:

    * The point's size should be the maximum temperature for the city

    * The point's color should be the city's name

    * Use the `hover_cols` parameter to the the "Hotel Name", "Country", and "Current Description" columns to each point as additional information.

6. From the map, *choose four cities* that a customer might want to visit. They should be close together and in the same country. Use the `loc` method to create separate DataFrames for each city on the travel route.

    > **Hint:** You will start and end the route in the same city, so the `vacation_start` and `vacation_end` DataFrames will be in the same city.

7. Use the Pandas `concat` function to merge the DataFrame from each city in the itinerary to create a new DataFrame named `itinerary_df` to store the itinerary details.

8. Use the Pandas `copy` function to create a new DataFrame named `waypoints_df` to store the longitude and latitude for each city in `itinerary_df`.

    > **Hint:** You'll use this DataFrame to create a map using GeoViews, so recall that the first column should be the longitude, and the second the latitude.

9. Use GeoViews to create a map that shows the four cities in the itinerary.

10. Next, you'll use the Geoapify Routing API to find a route between the cities in the itinerary. Review the code that sets the initial parameters and fetches the coordinates from each city to define the `waypoints` parameter by using a `for` loop.

    > **Hint:** You can note that the `mode` parameter is set to `drive`, you can play around with other modes as it's shown in [the "Travel modes" table](https://apidocs.geoapify.com/docs/routing/#api) in the Geoapify Routing API documentation.

11. Use the Geoapify Routing API to retrieve the route's directions for your itinerary.

12. From the JSON response, store the route's legs coordinates in a variable called `legs`.

13. Loop through the route legs coordinates to fetch the latitude and longitude for each step. Store the latitude and longitude values into two Python lists named `longitude` and `latitude`.

14. Use the `longitude` and `latitude` Python lists to create a new DataFrame named `route_df`.

15. Use the GeoViews `Path` function to configure a line plot by using `route_df`. Set a custom color and width for the line that may contrast with the map.

16. Use the asterisk operator to display a composed plot that shows the itinerary's route over the map containing the cities. 

17. Save your map to the `Vacation_Itinerary` folder as `WeatherPy_travel_map.png`.

---

## Make sure the initial dependencies and the Geoapify API key are imported

In [149]:
# Dependencies and Setup
import geoviews as gv
import hvplot.pandas
import pandas as pd
import requests
import os

# Turn off warning messages
import warnings
warnings.filterwarnings("ignore")

# Import API key
from config import geoapify_key

## From your `Vacation_Search` folder from Deliverable 2, import the `WeatherPy_vacation.csv` file as a DataFrame named `vacation_df`

In [2]:
# Read the WeatherPy_vacation.csv into a DataFrame
vacation_data=os.path.join(".././Vacation_Search/WeatherPy_vacation.csv")
vacation_df =pd.read_csv(vacation_data)

# Display sample data
vacation_df.head()

,City_ID,City,Country,Max Temp,Current Condition,Lat,Lng,Hotel Name
0,7,Sao Jose De Piranhas,BR,86.54,overcast clouds,-7.1206,-38.5019,Hotel Pousada São José
1,17,Chuy,UY,74.46,clear sky,-33.6971,-53.4616,Alerces
2,20,Sao Felix Do Xingu,BR,70.21,heavy intensity rain,-6.6447,-51.9950,Hotel Terraço
3,21,Mar Del Plata,AR,86.83,overcast clouds,-38.0023,-57.5575,Nuevo Ostende
4,27,Souillac,MU,76.17,overcast clouds,-20.5167,57.5167,Hotel SkylineInn


## Use GeoViews to create a map that shows all the cities in the `vacation_df` DataFrame. Configure the map as follows:

* The point's size should be the maximum temperature for the city

* The point's color should be the city's name

* Use the `hover_cols` parameter to the the "Hotel Name", "Country", and "Current Description" columns to each point as additional information.

In [3]:
# Configure the map plot
map_plot=vacation_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles="OSM",
    size="Max Temp",
    color= "City",
    alpha=0.5,
    frame_width = 600,
    frame_height = 500,
    hover_cols = ["Hotel Name","Current Condition", "Country"])

map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Max Temp,Hotel Name,Current Condition,Country)

## From the map, *choose four cities* that a customer might want to visit. They should be close together and in the same country. Use the `loc` method to create separate DataFrames for each city on the travel route.

In [4]:
# Create DataFrames for each city by filtering the 'vacation_df' using the loc method
vacation_start =vacation_df.loc[(vacation_df["City"]== "Rawson")]
vacation_end = vacation_df.loc[(vacation_df["City"]== "Rawson")]
vacation_stop1 =vacation_df.loc[(vacation_df["City"]== "Mar Del Plata")]
vacation_stop2 = vacation_df.loc[(vacation_df["City"]== "Rocha")]
vacation_stop3 = vacation_df.loc[(vacation_df["City"]== "Valparaiso")]

## Use the Pandas `concat` function to merge the DataFrame from each city in the itinerary to create a new DataFrame named `itinerary_df` to store the itinerary details

In [5]:
# Use the Pandas pd.concat() function to create a new DataFrame to store the itinerary details.
itinerary_df = pd.concat([vacation_start,vacation_end,vacation_stop1,
                         vacation_stop2,vacation_stop3])

# Display sample data
itinerary_df

,City_ID,City,Country,Max Temp,Current Condition,Lat,Lng,Hotel Name
87,400,Rawson,AR,86.29,clear sky,-43.3002,-65.1023,Hotel Deportivo
87,400,Rawson,AR,86.29,clear sky,-43.3002,-65.1023,Hotel Deportivo
3,21,Mar Del Plata,AR,86.83,overcast clouds,-38.0023,-57.5575,Nuevo Ostende
66,293,Rocha,UY,79.21,clear sky,-34.4833,-54.3333,Hotel Municipal
148,652,Valparaiso,CL,82.63,clear sky,-33.0393,-71.6273,Ibis Hotel


## Use the Pandas `copy` function to create a new DataFrame named `waypoints_df` to store the longitude and latitude for each city in `itinerary_df`

In [10]:
# Create a Pandas DataFrame to store the latitude and longitude for each city in the itineray
#how to use .copy() df2 = df1[["","","",""]].copy()
waypoints_df = itinerary_df[["City","Lat","Lng"]].copy()

# Display sample data
waypoints_df

,City,Lat,Lng
87,Rawson,-43.3002,-65.1023
87,Rawson,-43.3002,-65.1023
3,Mar Del Plata,-38.0023,-57.5575
66,Rocha,-34.4833,-54.3333
148,Valparaiso,-33.0393,-71.6273


## Use GeoViews to create map that shows the four cities in the itinerary

In [7]:
# Configure the map plot by using the itineraty_df
waypoints_map = waypoints_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles= "OSM",
    color = "City"
)


In [8]:
# Display the route_map
waypoints_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City)

## Next, you'll use the Geoapify Routing API to find a route between the cities in the itinerary. Review the code that sets the initial parameters and fetches the coordinates from each city to define the `waypoints` parameter by using a `for` loop

In [11]:
# Set parameters to trace the route
radius = 5000
params = {
    "mode":"drive",
    "apiKey": geoapify_key,
}

In [12]:
# Set an empty waypoints String variable
waypoints = ""

# Iterate through the route_df DataFrame to define the waypoints
for index, row in waypoints_df.iterrows():
    waypoints = waypoints + str(row["Lat"]) + "," + str(row["Lng"]) + "|"

# Delete the last character from the string
waypoints = waypoints[:-1]

# Add the waypoints to the params dictionary
params["waypoints"] = waypoints

# Display the params dictionary
params

{'mode': 'drive',
 'apiKey': '062e8064f5164593bb8dd25552f423f9',
 'waypoints': '-43.3002,-65.1023|-43.3002,-65.1023|-38.0023,-57.5575|-34.4833,-54.3333|-33.0393,-71.6273'}

## Use the Geoapify Routing API to retrieve the route's directions for your itinerary

In [122]:
# Set up the base URL for the Geoapify Places API.
base_url = "https://api.geoapify.com/v1/routing"

# Make request and retrieve the JSON data by using the params dictionaty
route_response = requests.get(base_url, params)

# Convert the API response to JSON format
route_response = route_response.json()

route_response['features'][0]['geometry']['coordinates']

[[[-65.101908, -43.299911], [-65.101908, -43.299911]],
 [[-65.101908, -43.299911],
  [-65.102413, -43.299548],
  [-65.101485, -43.29889],
  [-65.101479, -43.298888],
  [-65.101451, -43.298871],
  [-65.101432, -43.298858],
  [-65.10138, -43.298841],
  [-65.101356, -43.298845],
  [-65.101331, -43.298847],
  [-65.101291, -43.298841],
  [-65.101256, -43.298827],
  [-65.101228, -43.298806],
  [-65.101211, -43.298779],
  [-65.101206, -43.29875],
  [-65.101212, -43.298726],
  [-65.101226, -43.298704],
  [-65.101247, -43.298685],
  [-65.101278, -43.298669],
  [-65.101314, -43.298661],
  [-65.101352, -43.29866],
  [-65.101422, -43.298626],
  [-65.101509, -43.298575],
  [-65.102345, -43.297961],
  [-65.103392, -43.297158],
  [-65.104163, -43.296589],
  [-65.104381, -43.296434],
  [-65.10448, -43.296358],
  [-65.105494, -43.295616],
  [-65.106526, -43.294861],
  [-65.106626, -43.294786],
  [-65.106886, -43.294591],
  [-65.107103, -43.294433],
  [-65.107753, -43.29396],
  [-65.108463, -43.293439],

## From the JSON response, store the route's legs coordinates in a variable called `legs`

In [123]:
# Fetch the route's legs coordinates from the JSON reponse
legs = route_response['features'][0]['geometry']['coordinates']

legs

[[[-65.101908, -43.299911], [-65.101908, -43.299911]],
 [[-65.101908, -43.299911],
  [-65.102413, -43.299548],
  [-65.101485, -43.29889],
  [-65.101479, -43.298888],
  [-65.101451, -43.298871],
  [-65.101432, -43.298858],
  [-65.10138, -43.298841],
  [-65.101356, -43.298845],
  [-65.101331, -43.298847],
  [-65.101291, -43.298841],
  [-65.101256, -43.298827],
  [-65.101228, -43.298806],
  [-65.101211, -43.298779],
  [-65.101206, -43.29875],
  [-65.101212, -43.298726],
  [-65.101226, -43.298704],
  [-65.101247, -43.298685],
  [-65.101278, -43.298669],
  [-65.101314, -43.298661],
  [-65.101352, -43.29866],
  [-65.101422, -43.298626],
  [-65.101509, -43.298575],
  [-65.102345, -43.297961],
  [-65.103392, -43.297158],
  [-65.104163, -43.296589],
  [-65.104381, -43.296434],
  [-65.10448, -43.296358],
  [-65.105494, -43.295616],
  [-65.106526, -43.294861],
  [-65.106626, -43.294786],
  [-65.106886, -43.294591],
  [-65.107103, -43.294433],
  [-65.107753, -43.29396],
  [-65.108463, -43.293439],

## Loop through the route legs coordinates to fetch the latitude and longitude for each step. Store the latitude and longitude value into two Python lists names `longitude` and `latitude`

In [139]:
# Create and empty list to store the longitude of each step
longitude = []

# Create and empty list to store the latitude of step
latitude = []

step=[]
# Loop through the legs coordinates to fetch the latitude and longitude for each step
for index in legs:
    length=len(index)
    
    for step in index:
        latitude.append(step[0])
        longitude.append(step[1])
longitude

lat_lngs=zip(latitude,longitude)

coordinates=list(lat_lngs)

coordinates

[(-65.101908, -43.299911),
 (-65.101908, -43.299911),
 (-65.101908, -43.299911),
 (-65.102413, -43.299548),
 (-65.101485, -43.29889),
 (-65.101479, -43.298888),
 (-65.101451, -43.298871),
 (-65.101432, -43.298858),
 (-65.10138, -43.298841),
 (-65.101356, -43.298845),
 (-65.101331, -43.298847),
 (-65.101291, -43.298841),
 (-65.101256, -43.298827),
 (-65.101228, -43.298806),
 (-65.101211, -43.298779),
 (-65.101206, -43.29875),
 (-65.101212, -43.298726),
 (-65.101226, -43.298704),
 (-65.101247, -43.298685),
 (-65.101278, -43.298669),
 (-65.101314, -43.298661),
 (-65.101352, -43.29866),
 (-65.101422, -43.298626),
 (-65.101509, -43.298575),
 (-65.102345, -43.297961),
 (-65.103392, -43.297158),
 (-65.104163, -43.296589),
 (-65.104381, -43.296434),
 (-65.10448, -43.296358),
 (-65.105494, -43.295616),
 (-65.106526, -43.294861),
 (-65.106626, -43.294786),
 (-65.106886, -43.294591),
 (-65.107103, -43.294433),
 (-65.107753, -43.29396),
 (-65.108463, -43.293439),
 (-65.108613, -43.29333),
 (-65.11

## Use the `longitude` and `latitude` Python lists to create a new DataFrame named `route_df`

In [175]:
lat_lngs=zip(latitude,longitude)

coordinates=list(lat_lngs)

coordinates


route_df = pd.DataFrame(coordinates,columns=["Lat","Lng"])

# Add the steps' longitude and latitude from each step as columns to the DataFrame
# YOUR CODE HERE

# Display sample data
route_df

,Lat,Lng
0,-65.101908,-43.299911
1,-65.101908,-43.299911
2,-65.101908,-43.299911
3,-65.102413,-43.299548
4,-65.101485,-43.298890
...,...,...
25435,-71.627160,-33.039554
25436,-71.627205,-33.039517
25437,-71.627405,-33.039380
25438,-71.627339,-33.039334


##  Use the GeoViews `Path` function to configure a line plot by using `route_df`. Set a custom color and width for the line that may contrast with the map

In [181]:
# Configure the route path by using the GeoViews' Path function
route_path = gv.Path(route_df).opts(color="gold",alpha=0.5,line_width=4)

route_path= waypoints_map * route_path

In [182]:
# Display a composed plot by using the route_map and route_path objects
display(route_path)

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City)
   .Path.I   :Path   [Lat,Lng]